# Adversarial - Robustness of image classification

    Copyright (C) 2020 Adrian Bevan, and 2023, 2024 Abbey Waldron
    and Alexander Booth Queen Mary University of London

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.
    
----------------------

In this notebook we will explore a method to generate an adversial example instance. Namely, we'll create an image that has been specifically designed to cause a model to make a mistake when classifying it despite resembling a valid input for a human.  

----------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from PIL import Image
import requests

In [ ]:
# read in an image
url = 'https://www.huntsmarine.com.au/cdn/shop/articles/620_Fishfisher_24_2048x2048.jpg?v=1701991259'
im = np.array(Image.open(requests.get(url, stream=True).raw), dtype=float)
im /= 255.


**Use tf.image.resize to resize the image to (224,224) as MobileNet will expect.  Plot the image.**

In [ ]:
im_resized = tf.image.resize(im, (224, 224))
plt.imshow(im_resized)

**We need to reshape the image too to feed into the MobileNet.**

In [ ]:
im_reshaped = np.expand_dims(im_resized, axis=0)
print(np.shape(im_reshaped))

In [ ]:
# get a pre-trained model that hopefully works on your image
from tensorflow.keras.applications import MobileNet

pre_trained_model = MobileNet(weights='imagenet')
# just to make sure you don't accidentally change these parameters
pre_trained_model.trainable = False

model = tf.keras.models.Sequential()
model.add(pre_trained_model)

**Use model.predict to get the predicted class, what is it?  What is the confidence of the prediction?**  Tip - you can find the image net class list e.g. here: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

In [ ]:
model.predict(im_reshaped)
# Target class.
np.argmax(model.predict(im_reshaped))
# Confidence
np.max(model.predict(im_reshaped))

In [ ]:
# now let's hack it
target_class = 849 # teapot
max_iter = 100
eps = 0.001
target_class_as_array = np.array([target_class])


The code in the following cell gives an outline of a fast gradient descent attack.  Give it the right inputs so you can use it on the image you found previously.  You need to make some changes in the code to make it work!

In [ ]:
# CODE IN THIS CELL DOES NOT WORK YET - MAKE IT WORK!

current_confidence = 0.0

#### remember to define the image_array
image_array = im_reshaped

i = 0
im_hacked = fast_gradient_method(model,image_array,eps,np.inf,y=target_class_as_array,targeted=True)
while (current_confidence < 0.9) and (i < max_iter):
    im_hacked = fast_gradient_method(model,im_hacked,eps,np.inf,y=target_class_as_array,targeted=True)
    ##### UPDATE THE CURRENT CONFIDENCE IN HERE ##########
    current_confidence = model.predict(im_hacked)[0][target_class]
    ######################################################
    print(current_confidence)
    i += 1

**What is the predicted class of the image now?  What is the confidence?**

In [ ]:
# New predicted class.
np.argmax(model.predict(im_hacked))

**Plot the new version of the image.  Can you tell the difference?**

In [ ]:
plt.imshow(im_hacked[0])

**Now plot the difference between the original and new images, you may need to multiply it be 100 if you can't see anything**

In [ ]:
plt.imshow((im_hacked[0]-im_try[0]))

In [ ]:
plt.imshow((im_hacked[0]-im_try[0])*100)

**What happens when you change the parameter eps?  Remember you might need to change max_iter if you hit it**